In [17]:
# Import libraries
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from array import *
from scipy.io import wavfile as wav
from scipy.signal import spectrogram

In [3]:
# Read in Bad Romance as numpy array, sum left-right channels
fs, data = wav.read('BadRomance.wav')
data_sum = np.sum(data, 1)

In [53]:
# Take one-sided fft of Bad Romance
f, t, Sxx = spectrogram(data_sum, fs=44100, nperseg = 2048, noverlap=1024, return_onesided=True, mode='magnitude')
Sxx = Sxx/2048
Sxx_filter = Sxx
amp_threshold = Sxx_filter < np.amax(Sxx)/16
Sxx_filter[amp_threshold] = 0

incoming_array = Sxx_filter[:,:215]


In [55]:
'''# Plot
plt.figure(figsize=(15,8))
plt.pcolormesh(t,f, Sxx)
plt.ylim([0,2000])
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram of Lady Gaga\'s "Bad Romance"', fontsize=20)
plt.show()'''

'# Plot\nplt.figure(figsize=(15,8))\nplt.pcolormesh(t,f, Sxx)\nplt.ylim([0,2000])\nplt.ylabel(\'Frequency [Hz]\')\nplt.xlabel(\'Time [sec]\')\nplt.title(\'Spectrogram of Lady Gaga\'s "Bad Romance"\', fontsize=20)\nplt.show()'

In [56]:
#analyze a single row of the incoming array
def look_at_row(row):
    non_zero1 = np.where(np.array(row) > 0.0)  #single out the indexes of all the non-zero indexes
    non_zero = non_zero1[0]   # separate out just the index array
    len_nz = len(non_zero)    # find length of the non-zero index array
    if len_nz > 0:
        element = row[non_zero[0]]   # save the information of the first element in 'bit'
        bit = [element]
        the_return = [non_zero[0]]
    else:
        bit = []
        the_return = []
    for i in range(len_nz-1):
        diff = non_zero[i+1] - non_zero[i]   # check to see if next element is a neighboring non-zero element
        if diff == 1:
            element = row[non_zero[i+1]]   # if it is a neighbor, save its information into 'bit'
            bit.append(element)
        else:
            the_return.append(bit)         # if it is not a neighbor, break the previous 'bit' and make a new one
            element = row[non_zero[i+1]]
            bit = [element]
            the_return.append(non_zero[i+1])
    the_return.append(bit)                 # save the final bit
    return(the_return)
        

#look at the whole incoming array
def make_one_array(incoming_array):
    final_array = []
    numrows = len(incoming_array)   
    for n in range(numrows):
        row_info = look_at_row(incoming_array[n])   # break down each row individually
        new_array = []
        length = len(row_info)
        b = 0
        while b <= (length-2):                  # save the relevant information to a piece of the final array
            this_piece = [n, row_info[b]]
            b += 1
            this_piece.append(max(row_info[b]))   # save the max volume to the piece
            this_piece.append(len(row_info[b]))   # save the count to the piece
            b += 1
            final_array.append(this_piece)   # add the piece to the final array
    return(final_array)
            

final_array = make_one_array(incoming_array)
#[row, column, count, max_volume]